# Apach Spark - RDD 자료구조

- RDD = Resilient Distributed Dataset
    * Resilient 사전적 의미 : '탄력 있는'
    * 클러스의 노드들 간에 공유되어서 병렬척으로 연산되는 자료구조
- Spark는 2가지 종류의 shared var을 제공
    * ``broadcast var``: 모든 노드들의 메모리에서 값을 저장하기 위해 사용됨
    * ``accumulators``: count나 sum과 같이 add되는 변수들을 의미

## Spark 시작하기

- 가장 먼저 하는 것은 ``SparkContext``객체를 생성. 이것은 스파크가 클러스터에 접근하는 방법을 알려준다. 
- ``SparkContext``를 생성하기 위해서는 실행할 app에 대한 정보를 포함하고 있는 ``SparkConf`` 객체를 먼저 만들어주어야 함

In [9]:
from pyspark import SparkContext, SparkConf

- ``SparkContext``는 한 번에 여러개를 run시킬 수 없기 때문에 기존의 SparkContext를 가져와야 함! => ``SparkContext.getOrCreate()``사용!
<br>
- appName : 클러스터 UI에 보여질 너의 app이름
- master : 내 로컬을 의미

In [10]:
conf = SparkConf().setAppName('appName').setMaster('master')
sc = SparkContext.getOrCreate()

## RDD

- RDD를 만드는 방법
    * parallelizing을 이용해 driver program 내부에 직접 만들기
    * 외부 스토리지 시스템안에 있는 데이터셋을 참조해 불러오기

### parallelize 방법

In [7]:
data = [1, 2, 3, 4, 5]
distData = sc.parallelize(data)
print(distData)

ParallelCollectionRDD[1] at readRDDFromFile at PythonRDD.scala:262


- parallelize로 만들어진 ``distData``는 이제 병렬 연산이 가능하다.
    * 예로 reduce와 lambda를 이용해 리스트의 누적합 구하기 가능

In [8]:
res = distData.reduce(lambda a, b: a + b)
print(res)

15


- 여기서 한 가지 중요한 파라미터는 ``partitions(=slices)``의 개수인데, 스파크는 클러스터의 각 파티션에서 하나의 작업을 수행한다. 보통은 클러스터에서 각 CPU당 2~4개의 파티션을 원한다. 일반적으로 스파크는 자동적으로 파티션의 개수를 너의 클러스테 맞게 세팅해준다. 그러나 사용자가 직접 수동적으로 파라미터를 집어넣어 파티션 개수를 설정해줄 수 있다. 이 파라미터는 두번째 인자에 들어감
    * ex) ``sc.parallelize(data, 10)`` -> 10개의 파티션!

### External datasets

- pyspark는 로컬 파일 시스템, HDFS, 카산드라, HBase, 아마존 S3 등과 같은 하둡으로 지원되는 스토리지로부터 분산 데이터셋을 만들 수 있다.

In [19]:
distFile = sc.textFile('./people.txt')
# map과 reduce이용해 txt파일 내부 각 라인마다 length 합산 가능
distFile.map(lambda l: len(l)).reduce(lambda a, b: a + b)

29

- ``textFile``도 두 번째 인자로 파티션 개수를 인자로 넣어줄 수 있음!

- ``SparkContext.wholeTextFiles``: 용량이 작은 여러 텍스트 파일들을 포함하는 디렉토리를 읽게하여 (filename, content) 쌍으로 결과값을 반환 가능
    * ``textFile``은 단순이 하나의 파일 내부의 1개의 라인을 1개의 row로 반환
- ``RDD.saveAsPickleFile``, ``SparkContext.pickleFile``은 RDD파일은 Pickle 형태의 파이썬 객체로 저장하거나 읽어올 수 있음
<br><br>
- 하지만 참고로 위와 같은 파일 로드 방법은 이후 Spark SQL의 ``read/write``메소드에 의해 대체되며 그 방법들이 더 선호됨..

### RDD Operations
- transformations 연산: 기존의 데이터에서 새로운 데이터셋을 만들 때
    * ``map`` : 특정 함수를 이용해 데이터셋 값들을 변환 후 새로운 RDD 자료구조로 반환
    * transformations는 스파크에서 lazy함. 왜? 연산을 즉시 수행하지 않음. 대신 어떤 변환을 취할지 기억함. actions가 driver program에게 반환할 결과를 요구할 때 연산 시작함. => 이런 프로세스는 효율적! 왜냐하면 ``map``이라는 변환은 ``reduce``라는 액션이 취할 때 수행하며 결국 ``reduce``의 값만 driver program에게 반환하고 ``map``이 반환하는 메모리를 많이 차지하는 큰 데이터셋을 반환하지 않아 메모리 절약이 가능
<br><br>
- actions 연산: 데이터셋에서 특정한 연산을 수행 후 driver program에게 value를 반환
    * ``reduce``: 특정 함수를 사용해 RDD 자료의 값들을 집계하고 최종 값을 driver program에게 반환
<br><br>

- 기본적으로 변환된 RDD는 액션을 수행할 때마다 재 연산이된다. 하지만 ``persist(or cache)``메소드를 이용해서 메모리에 RDD을 유지시킬 수 있다. 이렇게 하는 이유는 다음번에 쿼리할 시 RDD에 더 빨리 접근하기 위해서 RDD들을 유지시키는 것임

In [20]:
# lines는 현재 메모리에 로드되지 않고 해당 파일을 가르키는 포인터임
lines = sc.textFile('people.txt')
# map이라는 변환을 취한 후의 결과값(연산되지 않은 상태)
lineLengths = lines.map(lambda s: len(s))
# reduce라는 액션을 취함으로써 병렬 처리를 하면서 작업 연산을 수행. 결과값만 driver program에게 반환!
totalLength = lineLengths.reduce(lambda a, b: a + b)

In [21]:
# 만약 변환을 취한 linesLengths를 추후에도 사용할 것이라면 persist로 저장
lineLengths.persist()

PythonRDD[8] at RDD at PythonRDD.scala:53

### Spark에 함수 pass하기

In [22]:
# Example 1 for using Class
class Myclass:
    def func(self, s):
        return s
    # Class내에 정의된 함수를 map 변환에 취해주기
    def doStuff(self, rdd):
        return rdd.map(self.func)

In [24]:
# Example 2 for using Class
# 생성자 함수의 인스턴스 변수를 이용
class Myclass:
    def __init__(self):
        self.field = 'Hello'
    def doStuff(self, rdd):
        return rdd.map(lambda s: self.field + s)

In [25]:
# Example 3 for using Class
# 위 방법이 헷갈리면 그냥 한 함수안에 변수까지 다 넣기
class Myclass:
    def __init__(self):
        self.field = 'Hello'
    def doStuff(self, rdd):
        field = self.field
        return rdd.map(lambda s: field + s)

### Key-Value Pairs로 작업하기

- 몇 가지 특별안한 연산들 key-value로 이루어진 쌍의 RDD에서만 가능함
    * 그룹핑이나 key별로 value를 집계하는 것 처럼 shuffle 연산일 때!
    * 파이썬 튜플과 같이 되어 있는 RDD구조에 이러한 연산 적용 가능

In [30]:
# 텍스트 파일에서 라인이 얼마나 많은지 세기 위한 연산으로 사용되는 reduceByKey
lines = sc.textFile('people.txt')
pairs = lines.map(lambda s: (s, 1))
counts = pairs.reduceByKey(lambda a, b: a + b)
counts

PythonRDD[45] at RDD at PythonRDD.scala:53

- 이외에 ``sortByKey()``와 driver program에서 결과값을 다시 도로 back하여 list객체로 반환하는 ``collect()``도 존재